# Schedule Cruncher for QSC Quarterly Schedules

Going to practice using Fall 2024 .xlsx file.

In [1]:
%pip install openpyxl
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\maria\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\maria\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\maria\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd 
import numpy as np 
from datetime import datetime

In [3]:
def parse_time_sheet(file_name, sheet_name, start_column, end_column, start_row, end_row):
    
    weekly_schedule= pd.read_excel(
        file_name, 
        sheet_name=sheet_name, 
        usecols=f'{start_column}:{end_column}', 
        skiprows=start_row-1, 
        nrows=end_row-start_row
    )

    weekly_schedule.rename(columns={"Unnamed: 1": "Time"}, inplace=True) # unnamed column renamed to time
    # next we are assigning tutors to a time and date 
    weekly_schedule = weekly_schedule.melt(id_vars=["Time"], var_name="Day", value_name="Tutor")
    # remove rows with missing tutors or irrelevant data
    weekly_schedule.dropna(subset=["Tutor"], inplace=True)
    # make sure tutor names are in title case
    weekly_schedule["Tutor"] = weekly_schedule["Tutor"].str.strip().str.title()

    schedule_dict = weekly_schedule.to_dict('records')

    # prev_time will track what the last time was if the current is null
    prev_time = schedule_dict[0]['Time']
    for i, entry in enumerate(schedule_dict):
        current_time = entry['Time']

        if pd.isna(current_time):
            entry["Time"] = prev_time

        else:
            prev_time = current_time
            entry["Time"] = "COORDINATOR"

        
    schedule_df = pd.DataFrame(schedule_dict)

    schedule_df = schedule_df[
        ~(
            (schedule_df["Time"] == datetime.strptime("10:00:00", '%H:%M:%S').time()) & 
            (schedule_df["Day"] == "Friday")
        )
    ]

    schedule_df = schedule_df[
        schedule_df["Time"] != "COORDINATOR" 
    ]

    # Regex to remove the time portion
    schedule_df['Tutor'] = schedule_df['Tutor'].str.replace(r'\s*\(.*?\)', '', regex=True)

    return schedule_df


In [4]:
# load in the course coverage sheet
def parse_course_coverage(file_name, sheet_name, start_column, end_column, start_row, end_row):
    
    course_coverage = pd.read_excel(
        file_name, 
        sheet_name=sheet_name, 
        usecols=f'{start_column}:{end_column}', 
        skiprows=start_row-1, 
        nrows=end_row-start_row
    )
    
    course_coverage.rename(columns={"Unnamed: 0": "Name"}, inplace=True)
    course_coverage = course_coverage.loc[:, ~course_coverage.columns.str.contains("Unnamed", na=False)]

    # Combine column names with the first row values
    course_coverage.columns = [f"{value}: {col}" if pd.notna(value) and col != "Name" else col for col, value in zip(course_coverage.columns, course_coverage.iloc[0])]

    # Drop the first row as it's now part of the header
    course_coverage = course_coverage[1:].reset_index(drop=True)

    return course_coverage



## Notes

Make sure all tutors names are in the correct column i.e. if a tutor is working on Friday but listed in the Saturday column this will not work. Make sure it is moved to the Friday column as an extra row. 

The first row of each time must be listed as a coordinator.

All times in partheses are disregarded at this time. 

All course codes listed in the course coverage list must be of the format STMATH 124, with no special characters.

In [5]:
# Rename these variables as needed each quarter

excel_file_name = "ALC 2024 Fall Schedule.xlsx"
time_sheet_name = "QSC Master Schedule Fall 2024"
course_coverage_sheet_name = "Course Coverage"


In [6]:
# Set bounds for time sheet 

time_sheet_df = parse_time_sheet(
    file_name=excel_file_name,
    sheet_name=time_sheet_name,
    start_column="B",
    end_column="I",
    start_row=2,
    end_row=51
)

print(time_sheet_df.tail(20))

         Time     Day    Tutor
200  14:00:00  Friday      Ben
201  14:00:00  Friday  Baoming
202  14:00:00  Friday    Aaron
203  14:00:00  Friday   Mariah
205  15:00:00  Friday    Kyler
206  15:00:00  Friday      Ben
207  15:00:00  Friday    Aaron
208  15:00:00  Friday   Mariah
210  12:00:00  Sunday  Vikrant
211  12:00:00  Sunday    Kyler
212  12:00:00  Sunday  Peter C
214  13:00:00  Sunday  Vikrant
215  13:00:00  Sunday    Kyler
216  13:00:00  Sunday  Peter C
218  14:00:00  Sunday  Vikrant
219  14:00:00  Sunday     Lyla
220  14:00:00  Sunday  Peter C
222  15:00:00  Sunday  Vikrant
223  15:00:00  Sunday     Lyla
224  15:00:00  Sunday  Peter C


In [7]:
grouped_time_sheet_df = time_sheet_df.groupby(["Day", "Tutor"])["Time"].agg(list)
grouped_time_sheet_df = grouped_time_sheet_df.reset_index()
print(grouped_time_sheet_df.tail(20))

          Day    Tutor                                               Time
35    Tuesday  Ibrahim           [15:00:00, 16:00:00, 17:00:00, 18:00:00]
36    Tuesday    Kirby                               [13:00:00, 14:00:00]
37    Tuesday     Lyla  [13:00:00, 14:00:00, 15:00:00, 16:00:00, 17:00...
38    Tuesday   Mariah                     [10:00:00, 11:00:00, 12:00:00]
39    Tuesday   Nikhil  [10:00:00, 11:00:00, 12:00:00, 13:00:00, 14:00...
40    Tuesday  Peter M           [15:00:00, 16:00:00, 17:00:00, 18:00:00]
41    Tuesday   Willow           [13:00:00, 14:00:00, 15:00:00, 16:00:00]
42  Wednesday    Aaron                                         [10:00:00]
43  Wednesday  Abigail                               [16:00:00, 17:00:00]
44  Wednesday  Baoming           [10:00:00, 11:00:00, 12:00:00, 13:00:00]
45  Wednesday      Ben                                         [18:00:00]
46  Wednesday    Bruce  [10:00:00, 11:00:00, 12:00:00, 13:00:00, 14:00...
47  Wednesday   Danika           [11:0

In [8]:
# Set bounds for the course coverage sheet

course_coverage_df = parse_course_coverage(
    file_name=excel_file_name,
    sheet_name=course_coverage_sheet_name,
    start_column="A",
    end_column="EZ",
    start_row=2,
    end_row=23
)

print(course_coverage_df)

       Name BMATH 121: Algebraic Reasoning BMATH 122: Precalculus I  \
0     Aaron                            NaN                        x   
1   Abigail                              x                        x   
2     Aryan                            NaN                      NaN   
3   Baoming                              x                        x   
4       Ben                            NaN                      NaN   
5     Bruce                            NaN                      NaN   
6    Danika                              x                        x   
7     Henry                              x                        x   
8   Ibrahim                            NaN                      NaN   
9     Kirby                              x                        x   
10    Kyler                            NaN                      NaN   
11     Lyla                            NaN                      NaN   
12   Mariah                              x                        x   
13    

In [9]:
def create_time_ranges(times):
    time_ranges = []

    start_time = times[0]
    end_time = -1

    for i in range(1, len(times)):
        prev_time = times[i-1]
        curr_time = times[i]

        # compare prev_time to curr_time
        if curr_time - prev_time == 1:
            continue
        else:
            end_time = prev_time + 1
            time_ranges.append((start_time, end_time))
            start_time = curr_time

    if end_time < start_time:
        time_ranges.append((start_time, times[-1] + 1))

    return time_ranges

In [10]:
def convert_hour_to_time_label(hour):
    if hour == 12:
        return "12PM"
    elif hour == 24 or hour == 0:
        return "12AM"
    elif hour > 12:
        return f"{hour - 12}PM"
    else:
        return f"{hour}AM"

In [11]:
"""
What do we need? 

- Course code
- Course name
- All days of the week
- Time ranges for tutor coverage for each day
- Tutors who can tutor the course

"""
with pd.ExcelWriter("Tutoring_Schedule_By_Course.xlsx") as writer:
    for course in course_coverage_df.columns[1:]:
        course_availablity = course_coverage_df.loc[:, ["Name", course]]
        tutor_names = course_availablity[course_availablity[course] == 'x']['Name'].to_list()
        # print(tutor_names)
        parsed_course = [c.strip() for c in course.split(":")]
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Sunday']

        tutor_times_for_course = []
        for day in days:
            day_hours = []
            for tutor in tutor_names:
                time_list = grouped_time_sheet_df[
                    (
                        (grouped_time_sheet_df["Day"] == day) &
                        (grouped_time_sheet_df["Tutor"] == tutor)
                    )
                ]

                if not time_list.empty:
                    day_hours += time_list["Time"].iloc[0]  # Append the list of times
                # get all times for each tutor on each day
            day_hours = list(set(t.hour for t in day_hours))
            day_hours.sort()

            time_ranges = create_time_ranges(day_hours) if len(day_hours) > 0 else [] 

            time_ranges = [f"{convert_hour_to_time_label(start)}-{convert_hour_to_time_label(end)}" for start, end in time_ranges]

            tutor_times_for_course.append(time_ranges)


        course_output = [
            parsed_course,
            *[[days[i]] + tutor_times_for_course[i] for i in range(len(days))],
            [],
            tutor_names
        ]

        course_df = pd.DataFrame(course_output)
    
        course_df.to_excel(writer, sheet_name=parsed_course[0], index=False, header=False)

    

